In [1]:
import numpy as np
import torch

from tqdm.auto import tqdm
from pathlib import Path

import itertools

import os
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv(), verbose=True)

True

In [2]:
!rm -r /home/huze/.cache/checkpoints

In [3]:
exp_configs = [p for p in Path('../../src/config/experiments/algonauts2021/').iterdir() if p.name.endswith('.yml')]

In [4]:
exp_configs

[PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_2d_segswin.yml'),
 PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_2d_simclr.yml'),
 PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_2d_pyconvsegnet.yml'),
 PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_2d_mobyswin.yml'),
 PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_2d_bdcnvgg.yml'),
 PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_3d_resnet.yml'),
 PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_2d_densnet.yml'),
 PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_2d_colorizer.yml'),
 PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_audio_vgg.yml'),
 PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_3d_flow.yml'),
 PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_3d_swin.yml')]

In [5]:
os.environ['RESULTS_DIR']

'/data/huze/ray_results/algonauts2021/'

In [6]:
from src.grid_runner import run_single_train
from src.grid_runner import run_single_tune_config
from src.config import get_cfg_defaults, combine_cfgs
from ray import tune

In [7]:
debug = False

In [8]:
analysis_list = []
for exp_config in exp_configs:
    cfg = combine_cfgs(
        path_cfg_data=exp_config,
        list_cfg_override=['DEBUG', debug]
    )
    
    name = exp_config.name.replace('.yml', '')

    # multi-layer 1 run
    analysis = tune.run(
        tune.with_parameters(
            run_single_tune_config,
            cfg=cfg
        ),
        config={
            'DATASET.ROI': tune.grid_search(['WB']),
            'TRAINER.CALLBACKS.BACKBONE.DEFROST_SCORE': tune.grid_search([1.]),
        },
        local_dir=cfg.RESULTS_DIR,
        resources_per_trial={"cpu": 4, "gpu": 1},
        name=name + '_multilayer_freeze_WB',
        verbose=1,
        resume='AUTO',
    )
    analysis_list.append(analysis)

2022-03-11 07:18:58,920	INFO tune.py:636 -- Total run time: 0.11 seconds (0.00 seconds for the tuning loop).


In [9]:
# load result df and find backbone deforest score

In [10]:
from src.utils.runs import load_run_df, filter_multi_layer_runs, filter_single_layer_runs, my_query_df

In [11]:
run_df = load_run_df(cfg.RESULTS_DIR)

  0%|          | 0/168 [00:00<?, ?it/s]

In [12]:
run_df = filter_multi_layer_runs(run_df)
run_df = my_query_df(run_df, equal_dict={'TRAINER.CALLBACKS.BACKBONE.DEFROST_SCORE': 1.0})

In [13]:
df = run_df[['MODEL.BACKBONE.NAME', 'DATASET.ROI', 'score']]

In [14]:
from src.utils.runs import mahou_list

In [15]:
defrost_df = df.copy()
minlestone_p = 0.5
defrost_df['defrost_score'] = defrost_df['score'].apply(lambda x: mahou_list(minlestone_p * x))

In [16]:
defrost_df

,MODEL.BACKBONE.NAME,DATASET.ROI,score,defrost_score
34,2d_bdcnvgg_warp_3d,WB,0.038811,0.018
37,2d_colorizer_warp_3d,WB,0.088229,0.045
54,2d_simclr_warp_3d,WB,0.135406,0.069
55,i3d_flow,WB,0.124815,0.063
89,i3d_rgb,WB,0.136212,0.069
90,3d_swin,WB,0.140836,0.069
123,audio_vggish,WB,0.058880,0.030
124,2d_pyconvsegnet_warp_3d,WB,0.134461,0.066
125,2d_densnet_warp_3d,WB,0.118351,0.060
144,2d_seg_swin_warp_3d,WB,0.115133,0.057


In [17]:
!mkdir ./config

mkdir: cannot create directory ‘./config’: File exists


E0311 07:19:38.657776286 3248497 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [18]:
defrost_df.to_csv('./config/defrost_df.csv')